In [23]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)
X = torch.rand(2, 20)
net(X)

tensor([[ 0.0371,  0.0623,  0.3714,  0.1704,  0.1046,  0.1931,  0.1315, -0.1541,
         -0.0453,  0.0490],
        [-0.0336,  0.1490,  0.3992,  0.1060,  0.2131,  0.1938,  0.0498, -0.1325,
         -0.0869, -0.0541]], grad_fn=<AddmmBackward0>)

自定义块
将输入数据作为其前向传播函数的参数。

通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。

计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。

存储和访问前向传播计算所需的参数。

根据需要初始化模型参数。

In [24]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的=初始化
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        # 我们定制的__init__函数通过super().__init__() 调用父类的__init__函数
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用Relu的函数版本，其在nn.functional模块中定义
        return self.out(F.relu((self.hidden(X))))

我们首先看一下前向传播函数，它以X作为输入， 计算带有激活函数的隐藏表示，并输出其未规范化的输出值。 在这个MLP实现中，两个层都是实例变量。 要了解这为什么是合理的，可以想象实例化两个多层感知机（net1和net2）， 并根据不同的数据对它们进行训练。 当然，我们希望它们学到两种不同的模型。

In [25]:
net = MLP()
net(X)

tensor([[ 0.1672,  0.0496,  0.0779, -0.1735, -0.1804, -0.0294, -0.1101, -0.2491,
         -0.1801,  0.0121],
        [ 0.0504,  0.0314,  0.0666,  0.0233,  0.0005, -0.0013,  0.0094, -0.1819,
         -0.0932, -0.0477]], grad_fn=<AddmmBackward0>)

In [27]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [28]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1324,  0.0588,  0.0249,  0.0499, -0.1279,  0.0714, -0.2228, -0.3792,
          0.1723, -0.0372],
        [ 0.1639,  0.0923,  0.0808,  0.0429, -0.0043,  0.1973, -0.1286, -0.2606,
          0.1678,  0.0222]], grad_fn=<AddmmBackward0>)

在前向传播中执行代码

In [29]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数，因此在训练期间保持不变
        self.rand_weight = torch.rand((20,20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 使用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [30]:
net = FixedHiddenMLP()
net(X)

tensor(0.0356, grad_fn=<SumBackward0>)

我们可以混合搭配各种组合块的方法。 在下面的例子中，我们以一些想到的方法嵌套块。

In [31]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(),
                        nn.Linear(16, 20),
                        FixedHiddenMLP())
chimera(X)

tensor(-0.1888, grad_fn=<SumBackward0>)